# Multi-label text classification using BERT

In [1]:
!nvidia-smi

Sat Jul 13 23:03:04 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   33C    P8             11W /  160W |     725MiB /  16384MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# !pip install transformers

## Imports

In [3]:
import os
from typing import List
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
import logging 

logging.basicConfig(
     level=logging.INFO, 
     format= '[%(asctime)s|%(levelname)s|%(module)s.py:%(lineno)s] %(message)s',
     datefmt='%H:%M:%S'
 )
import tqdm.notebook as tq
from tqdm import tqdm
# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()
from collections import defaultdict

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, AdamW, CamembertTokenizer, CamembertModel

from defi_textmine_2025.data import load_test_raw_data
from defi_textmine_2025.data import TARGET_COL, INTERIM_DIR, MODELS_DIR, submission_path

# BASE_CHECKPOINT = "bert-base-uncased"
# BASE_CHECKPOINT = "bert-base-multilingual-cased"
BASE_CHECKPOINT = "camembert/camembert-base"
TASK_NAME = "multilabel_tagged_text"

entity_classes = {'TERRORIST_OR_CRIMINAL', 'LASTNAME', 'LENGTH', 'NATURAL_CAUSES_DEATH', 'COLOR', 'STRIKE', 'DRUG_OPERATION', 'HEIGHT', 'INTERGOVERNMENTAL_ORGANISATION', 'TRAFFICKING', 'NON_MILITARY_GOVERNMENT_ORGANISATION', 'TIME_MIN', 'DEMONSTRATION', 'TIME_EXACT', 'FIRE', 'QUANTITY_MIN', 'MATERIEL', 'GATHERING', 'PLACE', 'CRIMINAL_ARREST', 'CBRN_EVENT', 'ECONOMICAL_CRISIS', 'ACCIDENT', 'LONGITUDE', 'BOMBING', 'MATERIAL_REFERENCE', 'WIDTH', 'FIRSTNAME', 'MILITARY_ORGANISATION', 'CIVILIAN', 'QUANTITY_MAX', 'CATEGORY', 'POLITICAL_VIOLENCE', 'EPIDEMIC', 'TIME_MAX', 'TIME_FUZZY', 'NATURAL_EVENT', 'SUICIDE', 'CIVIL_WAR_OUTBREAK', 'POLLUTION', 'ILLEGAL_CIVIL_DEMONSTRATION', 'NATIONALITY', 'GROUP_OF_INDIVIDUALS', 'QUANTITY_FUZZY', 'RIOT', 'WEIGHT', 'THEFT', 'MILITARY', 'NON_GOVERNMENTAL_ORGANISATION', 'LATITUDE', 'COUP_D_ETAT', 'ELECTION', 'HOOLIGANISM_TROUBLEMAKING', 'QUANTITY_EXACT', 'AGITATING_TROUBLE_MAKING'}
categories_to_check = ['END_DATE', 'GENDER_MALE', 'WEIGHS', 'DIED_IN', 'HAS_FAMILY_RELATIONSHIP', 'IS_DEAD_ON', 'IS_IN_CONTACT_WITH', 'HAS_CATEGORY', 'HAS_CONTROL_OVER', 'IS_BORN_IN', 'IS_OF_SIZE', 'HAS_LATITUDE', 'IS_PART_OF', 'IS_OF_NATIONALITY', 'IS_COOPERATING_WITH', 'DEATHS_NUMBER', 'HAS_FOR_HEIGHT', 'INITIATED', 'WAS_DISSOLVED_IN', 'HAS_COLOR', 'CREATED', 'IS_LOCATED_IN', 'WAS_CREATED_IN', 'IS_AT_ODDS_WITH', 'HAS_CONSEQUENCE', 'HAS_FOR_LENGTH', 'INJURED_NUMBER', 'START_DATE', 'STARTED_IN', 'GENDER_FEMALE', 'HAS_LONGITUDE', 'RESIDES_IN', 'HAS_FOR_WIDTH', 'IS_BORN_ON', 'HAS_QUANTITY', 'OPERATES_IN', 'IS_REGISTERED_AS']

mlb = MultiLabelBinarizer()
mlb.fit([categories_to_check])
logging.info(f"{mlb.classes_=}")

generated_data_dir_path = os.path.join(INTERIM_DIR, "multilabel_tagged_text_dataset")
assert os.path.exists(generated_data_dir_path)

preprocessed_data_dir = os.path.join(INTERIM_DIR, "one_hot_multilabel_tagged_text_dataset")
labeled_preprocessed_data_dir_path = os.path.join(preprocessed_data_dir,"train")
! mkdir -p {labeled_preprocessed_data_dir_path}

model_dir_path = os.path.join(MODELS_DIR, f"finetuned-{BASE_CHECKPOINT}")
! mkdir -p {model_dir_path}
model_dict_state_path = os.path.join(model_dir_path,"MLTC_model_state.bin")

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

[23:03:06|INFO|1676573907.py:44] mlb.classes_=array(['CREATED', 'DEATHS_NUMBER', 'DIED_IN', 'END_DATE', 'GENDER_FEMALE',
       'GENDER_MALE', 'HAS_CATEGORY', 'HAS_COLOR', 'HAS_CONSEQUENCE',
       'HAS_CONTROL_OVER', 'HAS_FAMILY_RELATIONSHIP', 'HAS_FOR_HEIGHT',
       'HAS_FOR_LENGTH', 'HAS_FOR_WIDTH', 'HAS_LATITUDE', 'HAS_LONGITUDE',
       'HAS_QUANTITY', 'INITIATED', 'INJURED_NUMBER', 'IS_AT_ODDS_WITH',
       'IS_BORN_IN', 'IS_BORN_ON', 'IS_COOPERATING_WITH', 'IS_DEAD_ON',
       'IS_IN_CONTACT_WITH', 'IS_LOCATED_IN', 'IS_OF_NATIONALITY',
       'IS_OF_SIZE', 'IS_PART_OF', 'IS_REGISTERED_AS', 'OPERATES_IN',
       'RESIDES_IN', 'STARTED_IN', 'START_DATE', 'WAS_CREATED_IN',
       'WAS_DISSOLVED_IN', 'WEIGHS'], dtype=object)


device(type='cuda')

In [4]:
def load_csv(dir_or_file_path: str, index_col=None, sep=',') -> pd.DataFrame:
    if os.path.isdir(dir_or_file_path):
        all_files = glob.glob(os.path.join(dir_or_file_path , "*.csv"))  
    else:
        assert dir_or_file_path.endswith(".csv")
        all_files = [dir_or_file_path]
    assert len(all_files) > 0
    return pd.concat([pd.read_csv(filename, index_col=index_col, header=0, sep=sep) for filename in all_files], axis=0, ignore_index=True)

def process_data(data: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([data, pd.DataFrame(mlb.transform(data[TARGET_COL]), columns=mlb.classes_, index=data.index)], axis=1) # .drop([TARGET_COL], axis=1)


def format_relations_str_to_list(labels_as_str: str) -> List[str]:
    return json.loads(
        labels_as_str.replace("{", "[").replace("}", "]").replace("'", '"')
    )  if not pd.isnull(labels_as_str) else []


def process_csv_to_csv(in_dir_or_file_path: str, out_dir_path: str) -> None:
    """Convert labels, i.e. list of relations category, into one-hot vectors

    Args:
        in_dir_or_file_path (str): str
        out_dir_path (str): str
    """
    if os.path.isdir(in_dir_or_file_path):
        all_files = glob.glob(os.path.join(in_dir_or_file_path , "*.csv"))  
    else:
        assert in_dir_or_file_path.endswith(".csv")
        all_files = [in_dir_or_file_path]
    for filename in (pb:=tqdm(all_files)):
        pb.set_description(filename)
        preprocessed_data_filename = os.path.join(out_dir_path, os.path.basename(filename))
        process_data(load_csv(filename).assign(**{TARGET_COL: lambda df: df[TARGET_COL].apply(format_relations_str_to_list)})).to_csv(preprocessed_data_filename, sep="\t")

## Preprocess and save data

- load generated data
- convert to dataframe
- convert categories into one-hot labels
- save into a tsv file

In [5]:
process_csv_to_csv(os.path.join(generated_data_dir_path, "train"), labeled_preprocessed_data_dir_path)

data/defi-text-mine-2025/interim/multilabel_tagged_text_dataset/train/41884.csv: 100%|██████████| 800/800 [00:04<00:00, 185.72it/s]


## Load preprocessed data

In [6]:
labeled_df = load_csv(labeled_preprocessed_data_dir_path, index_col=0, sep='\t')

In [7]:
labeled_df

,text_index,e1,e2,text,relations,CREATED,DEATHS_NUMBER,DIED_IN,END_DATE,GENDER_FEMALE,...,IS_OF_SIZE,IS_PART_OF,IS_REGISTERED_AS,OPERATES_IN,RESIDES_IN,STARTED_IN,START_DATE,WAS_CREATED_IN,WAS_DISSOLVED_IN,WEIGHS
0,2576,0,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2576,1,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2576,0,1,"Le matin du 10 janvier 2010, Arthur et Jacques...",['HAS_CONSEQUENCE'],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2576,1,1,"Le matin du 10 janvier 2010, Arthur et Jacques...",[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2576,2,0,"Le matin du 10 janvier 2010, Arthur et Jacques...",[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124000,41884,11,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124001,41884,12,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124002,41884,15,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124003,41884,17,22,"Le 14 janvier 2014, en pleine Sibérie, dans un...",['HAS_COLOR'],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Train-Validation split

- split such that each category exist at least in the train dataset

In [8]:
mlb.classes_

array(['CREATED', 'DEATHS_NUMBER', 'DIED_IN', 'END_DATE', 'GENDER_FEMALE',
       'GENDER_MALE', 'HAS_CATEGORY', 'HAS_COLOR', 'HAS_CONSEQUENCE',
       'HAS_CONTROL_OVER', 'HAS_FAMILY_RELATIONSHIP', 'HAS_FOR_HEIGHT',
       'HAS_FOR_LENGTH', 'HAS_FOR_WIDTH', 'HAS_LATITUDE', 'HAS_LONGITUDE',
       'HAS_QUANTITY', 'INITIATED', 'INJURED_NUMBER', 'IS_AT_ODDS_WITH',
       'IS_BORN_IN', 'IS_BORN_ON', 'IS_COOPERATING_WITH', 'IS_DEAD_ON',
       'IS_IN_CONTACT_WITH', 'IS_LOCATED_IN', 'IS_OF_NATIONALITY',
       'IS_OF_SIZE', 'IS_PART_OF', 'IS_REGISTERED_AS', 'OPERATES_IN',
       'RESIDES_IN', 'STARTED_IN', 'START_DATE', 'WAS_CREATED_IN',
       'WAS_DISSOLVED_IN', 'WEIGHS'], dtype=object)

In [9]:
labeled_df[mlb.classes_].sum(axis=0).sort_values()

HAS_LATITUDE                 10
HAS_LONGITUDE                12
HAS_FOR_HEIGHT               12
WAS_DISSOLVED_IN             14
HAS_FOR_WIDTH                14
WAS_CREATED_IN               15
HAS_FOR_LENGTH               16
IS_BORN_ON                   20
IS_REGISTERED_AS             34
WEIGHS                       41
DIED_IN                      41
IS_BORN_IN                   50
IS_DEAD_ON                   68
INJURED_NUMBER               70
DEATHS_NUMBER                75
HAS_COLOR                    91
CREATED                     126
IS_OF_NATIONALITY           179
HAS_QUANTITY                191
HAS_FAMILY_RELATIONSHIP     222
RESIDES_IN                  222
IS_COOPERATING_WITH         372
GENDER_FEMALE               414
IS_OF_SIZE                  438
INITIATED                   469
HAS_CONSEQUENCE             769
END_DATE                    874
HAS_CATEGORY                894
GENDER_MALE                 908
START_DATE                 1034
IS_PART_OF                 1462
IS_AT_OD

In [10]:
labeled_df.query("HAS_FOR_HEIGHT==1")[mlb.classes_].sum(axis=0)

CREATED                     0
DEATHS_NUMBER               0
DIED_IN                     0
END_DATE                    0
GENDER_FEMALE               0
GENDER_MALE                 0
HAS_CATEGORY                0
HAS_COLOR                   0
HAS_CONSEQUENCE             0
HAS_CONTROL_OVER            0
HAS_FAMILY_RELATIONSHIP     0
HAS_FOR_HEIGHT             12
HAS_FOR_LENGTH              0
HAS_FOR_WIDTH               0
HAS_LATITUDE                0
HAS_LONGITUDE               0
HAS_QUANTITY                0
INITIATED                   0
INJURED_NUMBER              0
IS_AT_ODDS_WITH             0
IS_BORN_IN                  0
IS_BORN_ON                  0
IS_COOPERATING_WITH         0
IS_DEAD_ON                  0
IS_IN_CONTACT_WITH          0
IS_LOCATED_IN               0
IS_OF_NATIONALITY           0
IS_OF_SIZE                  0
IS_PART_OF                  0
IS_REGISTERED_AS            0
OPERATES_IN                 0
RESIDES_IN                  0
STARTED_IN                  0
START_DATE

In [11]:
df_train, df_valid = train_test_split(labeled_df, test_size = 0.2, shuffle=True, random_state=42)
df_train.shape, df_valid.shape

((99204, 42), (24801, 42))

In [12]:
pd.DataFrame({"train": df_train[mlb.classes_].sum(axis=0), "valid": df_valid[mlb.classes_].sum(axis=0)}).sort_values("train", ascending=False)

,train,valid
IS_LOCATED_IN,7237,1788
HAS_CONTROL_OVER,3631,916
IS_IN_CONTACT_WITH,2324,595
OPERATES_IN,1914,521
STARTED_IN,1512,348
IS_AT_ODDS_WITH,1224,302
IS_PART_OF,1157,305
START_DATE,837,197
GENDER_MALE,723,185
HAS_CATEGORY,705,189


In [13]:
df_train_with_relation = df_train[df_train[mlb.classes_].sum(axis=1) >= 1]
df_train_with_relation.shape

(21030, 42)

In [14]:
df_train_without_relation = df_train[df_train[mlb.classes_].sum(axis=1) == 0]
df_train_without_relation.shape

(78174, 42)

## Create the tokenized datasets for model input

In [15]:
# Hyperparameters
MAX_LEN = 350 # TODO: increase
# tokenizer = BertTokenizer.from_pretrained(BASE_CHECKPOINT)
tokenizer = CamembertTokenizer.from_pretrained(BASE_CHECKPOINT)
task_special_tokens = ["<e1>", "</e1>", "<e2>", "</e2>"] + [
    f"<{entity_class}>" for entity_class in entity_classes
]
# add special tokens to the tokenizer
num_added_tokens = tokenizer.add_tokens(task_special_tokens, special_tokens=True)
num_added_tokens

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


59

In [16]:
# Test the tokenizer
test_text = "La <e2><NON_MILITARY_GOVERNMENT_ORGANISATION>police</e2> tchèque a <e2><NON_MILITARY_GOVERNMENT_ORGANISATION>mis la main</e2> sur le couple responsable d'un trafic d'œuvres d'art. Il s'agit de <e1><TERRORIST_OR_CRIMINAL>Patel</e1> et Mirna Maroski. Une <e2><NON_MILITARY_GOVERNMENT_ORGANISATION>perquisition</e2> à leur domicile a permis de retrouver une centaine de tableaux d'artistes européens. Il y avait également des pots en céramique et en porcelaine d'origine chinoise, ainsi que plusieurs faux documents de voyage. Les époux Maroski ont été conduits au poste de <e2><NON_MILITARY_GOVERNMENT_ORGANISATION>police</e2> dans un véhicule blindé. Mirna Maroski s'est évanouie une fois arrivée au poste. Elle a été amenée en ambulance au CHU de Motol où elle a été soignée. Monsieur Sergueï Alekseï, le directeur de l'hôpital, a demandé à ses collaborateurs d'être vigilants et de ne pas se laisser corrompre par la criminelle."
# generate encodings
encodings = tokenizer.encode_plus(test_text, 
                                  add_special_tokens = True,
                                  max_length = MAX_LEN,
                                  truncation = True,
                                  padding = "max_length", 
                                  return_attention_mask = True, 
                                  return_tensors = "pt")
# we get a dictionary with three keys (see: https://huggingface.co/transformers/glossary.html) 
encodings

{'input_ids': tensor([[    5,    61, 32007, 32044,  1642, 32008, 17499,    33, 32007, 32044,
           467,    13,   489, 32008,    32,    16,  1145,  1295,    18,    11,
            59,  3893,    18,    11,  2172,    10,    18,    11,   559,     9,
            69,    52,    11,   567,     8, 32005, 32035,  4844,   647, 32006,
            14, 11962,   778,  1525,   189,  5812,     9,   180, 32007, 32044,
         27417, 32008,    15,    97,  1711,    33,   994,     8,  1008,    28,
          6848,     8,  5705,    18,    11, 10407,  3980,     9,    69,   102,
           171,   200,    20, 13035,    22,  9504,    14,    22, 17533,    18,
            11,   870,  7880,     7,   163,    27,   247,  2601,  1801,     8,
           805,     9,    74,  9493,  1525,   189,  5812,    96,   101,  2653,
            10,    36,  1337,     8, 32007, 32044,  1642, 32008,    29,    23,
          1690, 25062,     9, 11962,   778,  1525,   189,  5812,    52,    11,
            41, 11506,   364,  3903,  

In [17]:
tokenizer.batch_decode(encodings['input_ids'])

["<s>La <e2> <NON_MILITARY_GOVERNMENT_ORGANISATION> police </e2> tchèque a <e2> <NON_MILITARY_GOVERNMENT_ORGANISATION> mis la main </e2> sur le couple responsable d'un trafic d'œuvres d'art. Il s'agit de <e1> <TERRORIST_OR_CRIMINAL> Patel </e1> et Mirna Maroski. Une <e2> <NON_MILITARY_GOVERNMENT_ORGANISATION> perquisition </e2> à leur domicile a permis de retrouver une centaine de tableaux d'artistes européens. Il y avait également des pots en céramique et en porcelaine d'origine chinoise, ainsi que plusieurs faux documents de voyage. Les époux Maroski ont été conduits au poste de <e2> <NON_MILITARY_GOVERNMENT_ORGANISATION> police </e2> dans un véhicule blindé. Mirna Maroski s'est évanouie une fois arrivée au poste. Elle a été amenée en ambulance au CHU de Motol où elle a été soignée. Monsieur Sergueï Alekseï, le directeur de l'hôpital, a demandé à ses collaborateurs d'être vigilants et de ne pas se laisser corrompre par la criminelle.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [18]:
len(tokenizer)

32064

In [19]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        # self.e1 = list(df['e1'])
        # self.e1 = list(df['e1'])
        # self.text_indexes = list(df['text_index'])
        self.title = list(df['text'])
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        text = str(self.title[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'title': text,
            # 'text_index': self.text_index[index],
            # 'e1': self.e1[index],
            # 'e2': self.e2[index],
        }

In [20]:
# most_common_categories = df_train[mlb.classes_].sum().sort_values(ascending=False).index[:7]
# logging.info(most_common_categories)
# # target_list = mlb.classes_.tolist()
# target_list = most_common_categories
target_list = mlb.classes_
logging.info(f"{len(target_list)} categories = {target_list}")

[23:03:14|INFO|1100424365.py:6] 37 categories = ['CREATED' 'DEATHS_NUMBER' 'DIED_IN' 'END_DATE' 'GENDER_FEMALE'
 'GENDER_MALE' 'HAS_CATEGORY' 'HAS_COLOR' 'HAS_CONSEQUENCE'
 'HAS_CONTROL_OVER' 'HAS_FAMILY_RELATIONSHIP' 'HAS_FOR_HEIGHT'
 'HAS_FOR_LENGTH' 'HAS_FOR_WIDTH' 'HAS_LATITUDE' 'HAS_LONGITUDE'
 'HAS_QUANTITY' 'INITIATED' 'INJURED_NUMBER' 'IS_AT_ODDS_WITH'
 'IS_BORN_IN' 'IS_BORN_ON' 'IS_COOPERATING_WITH' 'IS_DEAD_ON'
 'IS_IN_CONTACT_WITH' 'IS_LOCATED_IN' 'IS_OF_NATIONALITY' 'IS_OF_SIZE'
 'IS_PART_OF' 'IS_REGISTERED_AS' 'OPERATES_IN' 'RESIDES_IN' 'STARTED_IN'
 'START_DATE' 'WAS_CREATED_IN' 'WAS_DISSOLVED_IN' 'WEIGHS']


In [21]:
df_train.shape, df_valid.shape

((99204, 42), (24801, 42))

In [22]:
train_dataset = CustomDataset(df_train, tokenizer, MAX_LEN, target_list)
valid_dataset = CustomDataset(df_valid, tokenizer, MAX_LEN, target_list)

In [23]:
# testing the dataset
next(iter(train_dataset))

{'input_ids': tensor([    5,  2445,  8666,   185, 12686,     7,  1328,     8,    17,    12,
          1289, 15686,  3985,  3569,     7,    23,  2422,    12,    12,    36,
          9834,     7,    33,   101, 20867,    15,    28,  2207, 26540,    29,
            16,   161,    26,  2837,     8,    17,    12,  4771,    18,    12,
            59,    20,   580,     8,    17,    12,  1289,     9,    61,  2207,
          9082,    32,    19,    21, 14922,    10, 19331,    10,    20, 19867,
            31,    52, 15447,   176,   193,   274,    29,    16,   256,     9,
            74,  7574,   530,  8836,    10,     8, 14611,     7,     8,  2970,
             7,     8, 32005, 32020, 12817,    10, 32006,    21,     7,    20,
         27825,    10,    14,     8,   587,   867,    24,  2748,    20, 14613,
            10,    14,    19,  8139, 32007, 32058,    36,   307,     8,    13,
           656, 32008,    21,     9,   436,    96, 11952,     8,   187,  2321,
            14, 18085,    10,    66,   

## Create data loaders

In [24]:
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32

# Data loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

## Prepare the model to trained

In [25]:
class BERTClass(torch.nn.Module):
    def __init__(self, tokenizer: CamembertTokenizer):
        super(BERTClass, self).__init__()
        self.bert_model = CamembertModel.from_pretrained(BASE_CHECKPOINT, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, len(target_list))
        # if you want to add new tokens to the vocabulary, then in general you’ll need to resize the embedding layers with
        # Source https://discuss.huggingface.co/t/adding-new-tokens-while-preserving-tokenization-of-adjacent-tokens/12604
        self.bert_model.resize_token_embeddings(len(tokenizer))

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass(tokenizer)

# # Freezing BERT layers: (tested, weaker convergence)
# for param in model.bert_model.parameters():
#     param.requires_grad = False

model.to(device)

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BERTClass(
  (bert_model): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32064, 768)
      (position_embeddings): Embedding(514, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [26]:
# BCEWithLogitsLoss combines a Sigmoid layer and the BCELoss in one single class. 
# This version is more numerically stable than using a plain Sigmoid followed 
# by a BCELoss as, by combining the operations into one layer, 
# we take advantage of the log-sum-exp trick for numerical stability.
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [27]:
# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)         

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Function to tain the model

In [35]:
# Training of the model for one epoch
def train_model(training_loader, model, optimizer):
    predictions = []
    prediction_probs = []
    target_values = []
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to training mode (activate dropout, batch norm)
    model.train()
    # initialize the progress bar
    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader), 
                      leave=True, colour='steelblue')
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        # forward
        outputs = model(ids, mask, token_type_ids) # (batch,predict)=(32,37)
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        # training accuracy, apply sigmoid, round (apply thresh 0.5)
        # outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
        # targets = targets.cpu().detach().numpy()
        # correct_predictions += np.sum(outputs==targets)
        # num_samples += targets.size   # total number of elements in the 2D array
        outputs = torch.sigmoid(outputs).cpu().detach()
        # thresholding at 0.5
        preds = outputs.round()
        targets = targets.cpu().detach()
        correct_predictions += np.sum(preds.numpy()==targets.numpy())
        num_samples += targets.numpy().size   # total number of elements in the 2D array
        
        # thresholding at 0.5
        preds = outputs.round()        
        predictions.extend(preds)
        prediction_probs.extend(outputs)
        target_values.extend(targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()
        # Update progress bar
        #loop.set_description(f"")
        #loop.set_postfix(batch_loss=loss)

    # returning: trained model, model accuracy, mean loss
    predictions = torch.stack(predictions)
    prediction_probs = torch.stack(prediction_probs)
    target_values = torch.stack(target_values)    

    return model, float(correct_predictions)/num_samples, f1_score(target_values, predictions, average="macro", zero_division=0), np.mean(losses)
    # return model, float(correct_predictions)/num_samples, np.mean(losses)

# torch.cuda.empty_cache()
# train_model(train_data_loader, model, optimizer)

## Function to evaluate the model

In [36]:
def get_predictions(model, data_loader):
    """
    Outputs:
      predictions - 
    """
    model = model.eval()
    
    titles = []
    predictions = []
    prediction_probs = []
    target_values = []

    with torch.no_grad():
      for data in tqdm(data_loader, "training"):
        title = data["title"]
        ids = data["input_ids"].to(device, dtype = torch.long)
        mask = data["attention_mask"].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data["targets"].to(device, dtype = torch.float)
        
        outputs = model(ids, mask, token_type_ids)
        # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
        outputs = torch.sigmoid(outputs).detach().cpu()
        # thresholding at 0.5
        preds = outputs.round()
        targets = targets.detach().cpu()

        titles.extend(title)
        predictions.extend(preds)
        prediction_probs.extend(outputs)
        target_values.extend(targets)
    
    predictions = torch.stack(predictions)
    prediction_probs = torch.stack(prediction_probs)
    target_values = torch.stack(target_values)
    
    return titles, predictions, prediction_probs, target_values


def eval_model(validation_loader, model):    
    predictions = []
    prediction_probs = []
    target_values = []
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()

    with torch.no_grad():
        # for batch_idx, data in tqdm(enumerate(validation_loader, 0), "evaluating"):
        for data in tqdm(validation_loader, "evaluating"):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # validation accuracy
            # add sigmoid, for the training sigmoid is in BCEWithLogitsLoss
            outputs = torch.sigmoid(outputs).cpu().detach()
            # thresholding at 0.5
            preds = outputs.round()
            targets = targets.cpu().detach()
            correct_predictions += np.sum(preds.numpy()==targets.numpy())
            num_samples += targets.numpy().size   # total number of elements in the 2D array

            predictions.extend(preds)
            prediction_probs.extend(outputs)
            target_values.extend(targets)
    
    predictions = torch.stack(predictions)
    prediction_probs = torch.stack(prediction_probs)
    target_values = torch.stack(target_values)    

    return float(correct_predictions)/num_samples, f1_score(target_values, predictions, average="macro", zero_division=0), np.mean(losses)


# eval_model(train_data_loader, model)

## Model Training

In [37]:
EPOCHS = 20
# THRESHOLD = 0.5 # threshold for the sigmoid
PATIENCE = 4
n_not_better_steps = 0
history = defaultdict(list)
best_f1_macro = 0
# assert not os.path.exists(model_dict_state_path), "The trained model is already serialized at {model_dict_state_path}"

if not os.path.exists(os.path.dirname(model_dict_state_path)):
    os.makedirs(os.path.dirname(model_dict_state_path))

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}/{EPOCHS}')
    model, train_acc, train_f1_macro, train_loss = train_model(train_data_loader, model, optimizer)
    val_acc, val_f1_macro, val_loss = eval_model(val_data_loader, model)

    print(f'train_loss={train_loss:.4f}, val_loss={val_loss:.4f} train_f1_macro={train_f1_macro:.4f}, val_f1_macro={val_f1_macro:.4f}')

    history['train_acc'].append(train_acc)
    history['train_f1_macro'].append(train_f1_macro)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_f1_macro'].append(val_f1_macro)
    history['val_loss'].append(val_loss)
    # save the best model
    if val_f1_macro > best_f1_macro:
        torch.save(model.state_dict(), model_dict_state_path)
        best_f1_macro = val_f1_macro
        n_not_better_steps = 0
    else: # check for early stopping
        n_not_better_steps += 1
        if n_not_better_steps >= PATIENCE:
            break
    

Epoch 1/20


  0%|          | 0/3101 [00:00<?, ?it/s]

In [ ]:
plt.rcParams["figure.figsize"] = (10,7)
plt.plot(history['train_f1_macro'], label='train F1 macro')
plt.plot(history['val_f1_macro'], label='validation F1 macro')
plt.plot(history['train_loss'], label='train loss')
plt.plot(history['val_loss'], label='validation loss')
plt.title('Training history')
plt.ylabel('F1 macro / loss')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1])
plt.grid()

## Evaluation of the model

In [ ]:
# Loading pretrained model (best model)
model = BERTClass(tokenizer)
model.load_state_dict(torch.load(model_dict_state_path))
model = model.to(device)

In [ ]:
# Evaluate the model using the test data
# val_acc, val_f1_macro, val_loss = eval_model(val_data_loader, model)

In [ ]:
# The accuracy looks OK, similar to the validation accuracy
# The model generalizes well !
# val_acc

In [ ]:
titles, predictions, prediction_probs, target_values = get_predictions(model, val_data_loader)

In [ ]:
# Generate Classification Metrics
#
# note that the total support is greater than the number of samples
# some samples have multiple lables

print(classification_report(target_values, predictions, target_names=target_list, zero_division=0))

In [ ]:
# import seaborn as sns
# def show_confusion_matrix(confusion_matrix):
#     hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
#     hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
#     hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
#     plt.ylabel('True category')
#     plt.xlabel('Predicted category');

In [ ]:
# cm = confusion_matrix(target_values, predictions)
# df_cm = pd.DataFrame(cm, index=target_list, columns=target_list)
# show_confusion_matrix(df_cm)

## Prepare submission

In [ ]:
df_test = load_csv(os.path.join(generated_data_dir_path, "test")) #.drop(TARGET_COL, axis=1)
df_test

In [ ]:
# df_test.head().drop(TARGET_COL, axis=1).assign(**{cat: [0]*df_test.head().shape[0] for cat in target_list})

In [ ]:
test_dataset = CustomDataset(df_test.drop(TARGET_COL, axis=1).assign(**{cat: [0]*df_test.shape[0] for cat in target_list}), tokenizer, MAX_LEN, target_list)

In [ ]:
TEST_BATCH_SIZE = 32

test_data_loader = torch.utils.data.DataLoader(test_dataset, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
titles, predictions, prediction_probs, target_values = get_predictions(model, test_data_loader)

In [ ]:

ml_labeled_test_df = pd.concat(
    [
        df_test.drop(TARGET_COL, axis=1),
        pd.Series(
            mlb.inverse_transform(
                pd.DataFrame(predictions.numpy(), columns=target_list, index=df_test.index)[mlb.classes_].values
            ),
            name=TARGET_COL,
            index=df_test.index
        )
    ],
    axis=1
)
ml_labeled_test_df

In [ ]:
text_idx_to_relations = {
    text_index: [l[0] for l in group_df.drop(["text_index", "text"], axis=1)[group_df.relations.str.len()>0].apply(lambda row: [[row.iloc[0], r, row.iloc[1]] for r in row.iloc[-1]] if len(row.iloc[-1]) > 0 else [], axis=1).values.tolist()]
 for text_index, group_df in tqdm(ml_labeled_test_df.groupby("text_index"))
}

In [ ]:
text_idx_to_relations[13]

In [ ]:
test_index = load_test_raw_data().index
test_index

In [ ]:
submission_df = pd.DataFrame({"id": list(text_idx_to_relations.keys()), TARGET_COL: list(text_idx_to_relations.values())}).set_index("id").loc[load_test_raw_data().index]
submission_df = submission_df.assign(relations= submission_df.relations.map(lambda x: str(x).replace("'", '"')))
submission_df

In [ ]:
submission_df.to_csv(submission_path)

In [ ]:
print(f"titles:{len(titles)} \npredictions:{predictions.shape} \nprediction_probs:{prediction_probs.shape} \ntarget_values:{target_values.shape}")